In [1]:
import sys
sys.path.append('C:\\Users\\fgria\\AppData\\Local\\conda\\conda\\envs\\SDCND\\lib\\site-packages\\')
import cv2

In [2]:
import csv, os
import numpy as np
import sklearn
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Flatten, Dense, Conv2D, MaxPooling2D, Dropout, Activation, Lambda, Cropping2D
from keras.callbacks import TensorBoard
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

Using TensorFlow backend.


In [3]:
def process_image(image):
    #Convert to HSV to adjust brightness
    HSV_image = cv2.cvtColor(image,cv2.COLOR_RGB2HSV)

    #Add a constant so that it prevents the image from being completely dark
    random_bright = .25+np.random.uniform()

    #Apply the brightness reduction to the V channel
    HSV_image[:,:,2] = HSV_image[:,:,2]*random_bright

    #Convert to RBG again
    HSV_image = cv2.cvtColor(HSV_image,cv2.COLOR_HSV2RGB)

    #Cut image
    Cut_image = HSV_image[55:135, :, :]

    #Convert to np.array
    processed_image = Cut_image.astype(np.float32)
    return processed_image

In [4]:
def create_model():
    model = Sequential()
    #Normalize
    model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(80, 320,3)))

    model.add(Conv2D(24, 5,5,subsample=(2,2), activation='relu'))
    model.add(Conv2D(36, 5,5,subsample=(2,2), activation='relu'))
    model.add(Conv2D(48, 5,5,subsample=(2,2), activation='relu'))
    model.add(Conv2D(64, 3,3, activation='relu'))
    model.add(Dropout(rate=0.5))
    model.add(Conv2D(64, 3,3, activation='relu'))
    model.add(Flatten())
    model.add(Dense(100))
    model.add(Activation('relu'))
    model.add(Dropout(rate=0.5))
    model.add(Dense(50))
    model.add(Dense(10))
    model.add(Dense(1))
    model.compile(loss='mse', optimizer='adam')
    return model

In [5]:
def dataset():

    images = []
    steering_angles = []
    project_path = './dataset/'
    datalist = os.listdir(project_path)
    
    for datapath in datalist:
        lines = []
        print("Loading Data: " + datapath)
        with open(project_path + datapath + "/driving_log.csv",newline='') as csvfile:
            reader = csv.reader(csvfile, delimiter=',')
            for line in reader:
                lines.append(line)
        print("Process image")
        for line in lines:
            #Drop some 0 steering data
            if float(line[3])== 0.0:
                if np.random.choice([True, False]):
                    continue
            
            img_center_filepath = project_path + datapath + '/IMG/' + line[0].split('\\')[-1]
            img_left_filepath = project_path + datapath + '/IMG/' + line[1].split('\\')[-1]
            img_right_filepath = project_path + datapath + '/IMG/' + line[2].split('\\')[-1]

            img_center = process_image(cv2.imread(img_center_filepath))
            img_left = process_image(cv2.imread(img_left_filepath))
            img_right = process_image(cv2.imread(img_right_filepath))


            images.extend([img_center, img_left, img_right])

            
            steering_center = float(line[3])*1
            # create adjusted steering measurement for the side camera images
            correction = 0.25 # this is a parameter to tune
            steering_left = steering_center + correction
            steering_right = steering_center - correction
            steering_angles.extend([steering_center, steering_left, steering_right])


            #flip
            img_center_flipped = np.fliplr(img_center)
            img_left_flipped = np.fliplr(img_left)
            img_right_flipped = np.fliplr(img_right)

            steering_center_flipped = steering_center*-1.0
            steering_left_flipped = steering_left*-1.0
            steering_right_flipped = steering_right*-1.0
            
            if np.random.rand() <0.01:
                """
                f, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20,15))
                ax1.imshow(cv2.cvtColor(cv2.imread(img_center_filepath), cv2.COLOR_BGR2RGB))
                ax1.set_title('Original Image', fontsize=20)
                ax2.imshow(cv2.cvtColor(img_center.astype(np.uint8), cv2.COLOR_BGR2RGB))
                ax2.set_title('Random Bright and cut Image', fontsize=20)
                ax3.imshow(cv2.cvtColor(img_center_flipped.astype(np.uint8), cv2.COLOR_BGR2RGB))
                ax3.set_title('Fliped  Image', fontsize=20)
                """
                #Save image
                try:
                    mpimg.imsave("md_output/" + "Original_" + line[0].split('\\')[-1],cv2.cvtColor(cv2.imread(img_center_filepath), cv2.COLOR_BGR2RGB))
                    mpimg.imsave("md_output/" + "Random_Bright_Cut" + line[0].split('\\')[-1],cv2.cvtColor(img_center.astype(np.uint8), cv2.COLOR_BGR2RGB))
                    mpimg.imsave("md_output/" + "Random_Bright_Cut_flipped" + line[0].split('\\')[-1],cv2.cvtColor(img_center_flipped.astype(np.uint8), cv2.COLOR_BGR2RGB))
                except FileNotFoundError:
                    os.mkdir("./md_output")
                    mpimg.imsave("md_output/" + "Original_" + line[0].split('\\')[-1],cv2.cvtColor(cv2.imread(img_center_filepath), cv2.COLOR_BGR2RGB))
                    mpimg.imsave("md_output/" + "Random_Bright_Cut" + line[0].split('\\')[-1],cv2.cvtColor(img_center.astype(np.uint8), cv2.COLOR_BGR2RGB))
                    mpimg.imsave("md_output/" + "Random_Bright_Cut_flipped" + line[0].split('\\')[-1],cv2.cvtColor(img_center_flipped.astype(np.uint8), cv2.COLOR_BGR2RGB))
            images.extend([img_center_flipped, img_left_flipped, img_right_flipped])
            steering_angles.extend([steering_center_flipped, steering_left_flipped, steering_right_flipped])

    print("Dataset complete.")
    X_train = np.array(images)
    y_train = np.array(steering_angles)
    return X_train, y_train

In [6]:
if __name__ == "__main__":
    X_train, y_train = dataset()

Loading Data: data_crush
Process image
Loading Data: data_fix
Process image
Loading Data: data_smooth1
Process image
Loading Data: data_smooth2
Process image
Loading Data: data_smooth3
Process image
Loading Data: data_smooth4
Process image
Dataset complete.


In [7]:
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    session = tf.Session(config=config)
    tf.keras.backend.set_session(session)
    
    print("Create model")
    model = create_model()
    model.fit(X_train, y_train, validation_split=0.3, shuffle=True, epochs=10)
    #model.save('model_test2.h5')
    model.summary()

Create model


C:\Users\fgria\AppData\Local\conda\conda\envs\Crawler\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(24, (5, 5), activation="relu", strides=(2, 2))`
  
C:\Users\fgria\AppData\Local\conda\conda\envs\Crawler\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(36, (5, 5), activation="relu", strides=(2, 2))`
  import sys
C:\Users\fgria\AppData\Local\conda\conda\envs\Crawler\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (5, 5), activation="relu", strides=(2, 2))`
  
C:\Users\fgria\AppData\Local\conda\conda\envs\Crawler\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  if __name__ == '__main__':
C:\Users\fgria\AppData\Local\conda\conda\envs\Crawler\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Update yo

Train on 46225 samples, validate on 19811 samples
Epoch 1/10
46225/46225 [==============================] - 48s 1ms/step - loss: 0.0478 - val_loss: 0.0270
Epoch 2/10
46225/46225 [==============================] - 41s 891us/step - loss: 0.0416 - val_loss: 0.0323
Epoch 3/10
46225/46225 [==============================] - 40s 866us/step - loss: 0.0397 - val_loss: 0.0255
Epoch 4/10
46225/46225 [==============================] - 41s 881us/step - loss: 0.0375 - val_loss: 0.0260
Epoch 5/10
46225/46225 [==============================] - 40s 870us/step - loss: 0.0368 - val_loss: 0.0267
Epoch 6/10
46225/46225 [==============================] - 40s 856us/step - loss: 0.0350 - val_loss: 0.0249
Epoch 7/10
46225/46225 [==============================] - 40s 859us/step - loss: 0.0334 - val_loss: 0.0256
Epoch 8/10
46225/46225 [==============================] - 39s 842us/step - loss: 0.0326 - val_loss: 0.0258
Epoch 9/10
46225/46225 [==============================] - 39s 848us/step - loss: 0.0318 - val_lo